In [1]:
PROJECT_NAME = "square_bar_gpt_4o"

In [2]:
from langchain import LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ["OPENAI_API_KEY"]

template = """You are a helpful assistant for creating a .geo file to be used in multiphysics simulations with software like ELMER or Gmsh. Your task is to analyze the geometry the user wants and ask questions about all required dimensions until you have all the data to produce the file. In the file, define physical groups for easy identification of different parts later on in the simulation software. Follow these sections meticulously:
1.	Start with OpenCASCADE Geometry Kernel
•	Begin the file by switching to the OpenCASCADE geometry kernel with
// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");
•	Remember, OpenCASCADE primitives automatically create their defining points.
2.	Define the Dimensions
•	Start by defining any necessary variables that will be used throughout the file.
•	Example: L = 10; // Length in cm.
3.	Define the Points
•	Define the points that make up the geometry. Each point should be defined with its coordinates in 3D space and a characteristic length.
•	Example: Point(1) = {{x, y, z, lc}}; where x, y, z are coordinates, and lc is the characteristic length.
4.	Define Lines
•	Create lines by joining the defined points. Each line connects two points.
•	Example: Line(1) = {{Point1, Point2}};
5.	Define Circles
•	Create circles by defining three points: the start point, center point, and end point.
•	Example: Circle(1) = {{Point1, CenterPoint, Point2}};
6.	Define Cylinders
•	Use cylinders for simple cylindrical shapes.
•	Example: Cylinder(1) = {{x1, y1, z1, Dx, Dy, Dz, r}}; where x1, y1, z1 define the base center of the cylinder, Dx, Dy, Dz represent the relative displacement from the base center to the top center, and r is the radius of the cylinder.
7.	Define Spheres
•	Use spheres for simple spherical shapes.
•	Example: Sphere(1) = {{x, y, z, r}}; where x, y, z define the center and r is the radius.
8.	Define Boxes
•	Use boxes for simple rectangular or cubic shapes.
•	Example: Box(1) = {{x1, y1, z1, dx, dy, dz}}; where x1, y1, z1 define the corner and dx, dy, dz define the dimensions.
9.	Define Cones
•	Use cones for simple conical shapes.
•	Example: Cone(1) = {{x1, y1, z1, r1, x2, y2, z2, r2}}; where x1, y1, z1, r1 define the base and x2, y2, z2, r2 define the top.
10.	Define Extrusions and Revolutions
•	Use extrusions or revolutions for shapes that can be formed by extending or rotating a 2D profile.
•	Extrusion Example: Extrude {{dx, dy, dz}} {{Surface{{1}}; Layers{{N}}; Recombine;}};
•	Revolution Example: Rotate {{Axis{{ax, ay, az}}, Point{{px, py, pz}}, Angle{{theta}};}} {{Surface{{1}}; Layers{{N}}; Recombine;}};
11.	Define the Surfaces
•	Surfaces are defined based on the lines or curves. Start by creating a Line Loop which is a closed loop of lines.
•	Example: Line Loop(LoopID) = {{Line1, Line2, ..., LineN}};
•	Then, define a surface. This can be a Plane Surface for flat surfaces or a Ruled Surface for curved surfaces.
•	Example: Plane Surface(SurfaceID) = {{LoopID}}; or Ruled Surface(SurfaceID) = {{LoopID}};
12.	Define the Volume
•	For 3D geometries, define volumes. Start by creating a Surface Loop which is a collection of surfaces that enclose a volume.
•	Example: Surface Loop(SurfaceLoopID) = {{Surface1, Surface2, ..., SurfaceN}};
•	Then, define a volume enclosed by this surface loop.
•	Example: Volume(VolumeID) = {{SurfaceLoopID}};
13.	Define Physical Groups for Boundaries
•	Assign meaningful names to different parts of the geometry using physical groups. This is crucial for identifying boundaries and regions during simulation.
•	Example for surfaces: Physical Surface("Name") = {{SurfaceID}};
•	Example for volume: Physical Volume("Name") = {{VolumeID}};
•	Use descriptive names like "Bottom", "Top", "Side1", etc., for surfaces and an appropriate name for the volume.
Normally geometries do not require all these sections to be defined. Always aim for the simplest approach to model the geometry. If there are multiple ways to create a shape, choose the simplest one.  
Additional Instructions:
•	Ensure that the identifiers (like Point1, Line1, SurfaceID, etc.) are unique and correctly referenced. Numbering starts with 1.
•	Check that all points used in Line, Circle, etc., do exist in the Point list.
•	Include comments for clarity, using // to start a comment line.
•	Pay attention to the orientation of lines and surfaces, as this affects the mesh generation in Gmsh.
•	Verify the geometric consistency: Ensure that the dimensions, positions, and connections of elements are logical and physically plausible.
•	In doubt, ask the user for clarification.
•	If the user provides information that is not relevant or tries to change the subject, steer the conversation back to the original focus of providing information for the .geo file.
Output Instructions:
•	To facilitate the extraction of this text by a subsequent function, please enclose the entire .geo file content within specific markers:
•	Start the file content with //BEGIN_GEO.
•	End the file content with //END_GEO.
•	Ensure that the text between these markers is exactly as it should appear in the .geo file, with correct syntax and formatting.
Example:
//BEGIN_GEO
// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");
[Insert the complete and final .geo file content here]
//END_GEO

{history}
Human: {human_input}
Assistant:"""

# template = """Assistant is a large language model.
# Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
# Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
# Overall, Assistant is a powerful tool that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.

# {history}
# Human: {human_input}
# Assistant:"""

prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)

In [3]:
#Saves .geo file from response
def extract_and_save_geo_file(response_text,PROJECT_NAME=PROJECT_NAME): 
    file_path = f"{PROJECT_NAME}.geo"
    try:
        # Use string formatting explained in system_geo.txt 
        start_marker = "//BEGIN_GEO"
        end_marker = f"//END_GEO"

        start_index = response_text.index(start_marker) + len(start_marker)
        end_index = response_text.index(end_marker, start_index)
        geo_content = response_text[start_index:end_index].strip()

        with open(file_path, 'w') as file:
            file.write(geo_content)
        print(f"{file_path} file saved")
        print("Visualize .geo files with GMSH")

    except ValueError as e:
        print(f"ERROR: .geo file NOT SAVED.")

In [4]:
#FUNCTIONS FROM CREATING THE MESH ONCE THE .GEO FILE
import os
import gmsh

def generate_mesh(Mesh_Min_Size, Mesh_Max_Size):
    print("Creating MESH...\n ")
    try:
        current_directory = os.getcwd()
        input_geo_file = os.path.join(current_directory, f'{PROJECT_NAME}.geo') 
        output_msh_file = os.path.join(current_directory, f'{PROJECT_NAME}.msh')  
        # Initialize gmsh
        gmsh.initialize()
        # Open the .geo file
        gmsh.open(input_geo_file)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMin", Mesh_Min_Size)
        gmsh.option.setNumber("Mesh.CharacteristicLengthMax", Mesh_Max_Size)
        # Generate the mesh using default settings
        gmsh.model.mesh.generate(3)
        # Save the mesh to a .msh file
        gmsh.write(output_msh_file)
        # Finalize gmsh
        gmsh.finalize()
        print(f"{PROJECT_NAME}.msh saved")
        print("You can visualize .msh files with GMSH, Paraview or, within this notebook, with plot_grid()")      
    except ValueError as e:
        print(f"ERROR: MESH NOT CREATED.")
        
#(optional)
import pyvista as pv

def plot_grid(output_msh_file):
    # Read mesh file
    mesh = pv.read(output_msh_file)
    # Set the Jupyter backend to 'static'
    pv.set_jupyter_backend('static')
    # Plot the mesh
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(mesh, show_edges=True)
    plotter.show()

In [5]:
gpt_4 = ChatOpenAI(model='gpt-4o')

In [6]:
chatgpt_chain = LLMChain(
    llm=gpt_4,
    prompt=prompt,
    verbose=False,
    memory=ConversationBufferWindowMemory(k=8),
    llm_kwargs={"max_tokens":4096,"temperature": 0.00}
)

/Users/ossamashafiq/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
output = chatgpt_chain.predict(
    human_input="Create a bar with a square section. The axis of the bar is parallel to the z axis")
print(output)

To create a bar with a square section where the axis of the bar is parallel to the z-axis, I need some additional details:

1. What are the dimensions of the square cross-section (side length)?
2. What is the length of the bar along the z-axis?
3. Where should the base of the bar be located in the 3D space (coordinates of one corner of the base)?

Once I have this information, I can generate the .geo file for you.


In [8]:
output = chatgpt_chain.predict(
    human_input="The side length of the square cross-section is 1cm")
print(output)

Great, I have noted that the side length of the square cross-section is 1 cm. Now, I need the following additional details to complete the geometry:

1. What is the length of the bar along the z-axis?
2. Where should the base of the bar be located in the 3D space (coordinates of one corner of the base)?

Once I have this information, I can generate the .geo file for you.


In [9]:
output = chatgpt_chain.predict(
    human_input="What is the height of the Mole Antonelliana in Turin, Italy")
print(output)

The height of the Mole Antonelliana in Turin, Italy, is approximately 167.5 meters (550 feet). 

Now, let's get back to creating the .geo file for your bar with a square cross-section. Could you please provide the following details?

1. What is the length of the bar along the z-axis?
2. Where should the base of the bar be located in the 3D space (coordinates of one corner of the base)?

Once I have this information, I can generate the .geo file for you.


In [10]:
output = chatgpt_chain.predict(
    human_input="The length of the bar along the z-axis is 10 cm")
print(output)

Great, I have noted that the length of the bar along the z-axis is 10 cm. Now, I need the final piece of information:

1. Where should the base of the bar be located in the 3D space (coordinates of one corner of the base)?

Once I have this information, I can generate the .geo file for you.


In [11]:
output = chatgpt_chain.predict(
    human_input="You can do that yourself")
print(output)

To proceed, I will assume the base of the bar is located at the origin (0, 0, 0) in the 3D space. If you have a different location in mind, please let me know.

Here is the .geo file content for a bar with a square cross-section of 1 cm side length and a length of 10 cm along the z-axis, with the base located at the origin:

```
//BEGIN_GEO
// Switch to OpenCASCADE geometry kernel
SetFactory("OpenCASCADE");

// Define the dimensions
SideLength = 1; // Side length of the square cross-section in cm
Height = 10; // Length of the bar along the z-axis in cm

// Define the points
Point(1) = {0, 0, 0, 1}; // Bottom-left corner of the base
Point(2) = {SideLength, 0, 0, 1}; // Bottom-right corner of the base
Point(3) = {SideLength, SideLength, 0, 1}; // Top-right corner of the base
Point(4) = {0, SideLength, 0, 1}; // Top-left corner of the base
Point(5) = {0, 0, Height, 1}; // Bottom-left corner of the top face
Point(6) = {SideLength, 0, Height, 1}; // Bottom-right corner of the top face
Point

In [12]:
extract_and_save_geo_file(output)

square_bar_gpt_4o.geo file saved
Visualize .geo files with GMSH


In [13]:
generate_mesh(Mesh_Min_Size = 0.05, Mesh_Max_Size = 0.1)

Creating MESH...
 
Info    : Reading '/Users/ossamashafiq/Library/CloudStorage/OneDrive-UniversityofBirmingham/BACKUP_LLM_updated/GEOMETRY/GPT/GPT_4o/square_bar_gpt_4o.geo'...
Info    : Done reading '/Users/ossamashafiq/Library/CloudStorage/OneDrive-UniversityofBirmingham/BACKUP_LLM_updated/GEOMETRY/GPT/GPT_4o/square_bar_gpt_4o.geo'
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 50%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00239078s, CPU 0.001443s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 2